In [2]:
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, SQLContext, functions as F
from pyspark.sql.functions import *

mongo_uri = "mongodb://hadoop-vm.internal.cloudapp.net:27017/ca2"

# Spark version 3.2.3
# MongoDB version 6.0.5
# Java Version 11

# create a spark session
# Jars dependencies available in maven repository
# https://mvnrepository.com/search?q=mongodb-driver-sync
spark = SparkSession.builder \
    .appName('Tweets') \
    .config("spark.mongodb.read.connection.uri", mongo_uri) \
    .config("spark.mongodb.write.connection.uri", mongo_uri) \
    .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:10.1.1") \
    .config("spark.jars.packages", "org.mongodb:mongodb-driver-core:4.9.1") \
    .config("spark.jars.packages", "org.mongodb:mongodb-driver-sync:4.9.1") \
    .config("spark.jars.packages", "org.mongodb:bson:4.9.1") \
    .getOrCreate()

# read data from mongodb collection "tweets" into a dataframe "df"
df = spark.read \
    .format("mongodb") \
    .option("connection.uri", mongo_uri) \
    .option("database", "ca2") \
    .option("collection", "tweets") \
    .load()

df.printSchema()

root
 |-- _id: long (nullable = true)
 |-- coordinates: void (nullable = true)
 |-- text: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestamp_ms: long (nullable = true)



## Tweets Statistics

### Total count tweets by laboratory

In [3]:
# No need to compare with lower case as we are looking for exact match on hastags
def contains_hashtags(column, hashtags):
    condition = None
    for hashtag in hashtags:
        if condition is None:
            condition = column.contains(hashtag)
        else:
            condition = condition | column.contains(hashtag)
    return condition


In [4]:
laboratories = ['#Moderna', '#Pfizer', '#BioNTech', '#AstraZeneca', '#JohnsonAndJohnson', '#Janssen', '#Novavax', '#Sinovac', '#Sinopharm', '#Sanofi', '#GSK']

In [10]:
# Filtering tweets containing laboratories
df_lab = df.filter(contains_hashtags(col("text"), laboratories))

In [13]:
df_lab_monthly = df_lab.groupBy(   
    when(col("text").contains('#AstraZeneca'), "AstraZeneca").alias("laboratory"),
    date_trunc("month", "timestamp").alias("month")
).count() \
.groupBy("laboratory").agg(avg("count").alias("avg_tweets_per_month"))

In [14]:
df_lab_monthly.show()

+-----------+--------------------+
| laboratory|avg_tweets_per_month|
+-----------+--------------------+
|AstraZeneca|   636.9166666666666|
|       null|   778.5833333333334|
+-----------+--------------------+



# Create AstraZeneca collection

In [11]:
laboratory = "AstraZeneca"
df_lab = df.filter(lower(col("text")).contains(laboratory.lower()))
print(f"Total tweets for {laboratory} {df_lab.count()}")

3708

In [12]:
df_lab.write \
    .format("mongodb") \
    .mode("overwrite") \
    .option("connection.uri", mongo_uri) \
    .option("database", "ca2") \
    .option("collection", f"{laboratory.lower()}_tweets") \
    .save()

# Create Moderna collection

In [13]:
laboratory = "Moderna"
df_lab = df.filter(lower(col("text")).contains(laboratory.lower()))
print(f"Total tweets for {laboratory} {df_lab.count()}")

Total tweets for Moderna 8246


In [14]:
df_lab.write \
    .format("mongodb") \
    .mode("overwrite") \
    .option("connection.uri", mongo_uri) \
    .option("database", "ca2") \
    .option("collection", f"{laboratory.lower()}_tweets") \
    .save()

# Create topic tweet "Vaccin [ne, nation,]"

In [15]:
topic = "Vaccin"
df_vaccine = df.filter(lower(col("text")).contains(topic.lower()))
print(f"Total tweets for {topic} {df_vaccine.count()}")

Total tweets for Vaccin 96869


In [16]:
df_vaccine.write \
    .format("mongodb") \
    .mode("overwrite") \
    .option("connection.uri", mongo_uri) \
    .option("database", "ca2") \
    .option("collection", f"{topic.lower()}_tweets") \
    .save()